In [ ]:
# Cell 1 – εγκατάσταση βιβλιοθήκης
!pip install pyswisseph


In [ ]:
# Cell 2 – imports, σταθερές
import csv, json, os, urllib.request, shutil
import swisseph as swe
from datetime import datetime
drive.mount('/content/drive')  # ενεργοποίησέ το μόνο σε Colab

PLANETS = {
    "Sun": swe.SUN, "Moon": swe.MOON, "Mercury": swe.MERCURY,
    "Venus": swe.VENUS, "Mars": swe.MARS, "Jupiter": swe.JUPITER,
    "Saturn": swe.SATURN, "Uranus": swe.URANUS,
    "Neptune": swe.NEPTUNE, "Pluto": swe.PLUTO,
}

SE_URL  = "https://ftp.astro.com/pub/swisseph/ephe/sepl_18.se1"
SE_FILE = "/content/drive/MyDrive/Studies/sepl_18.se1"
INPUT_FILE = "/content/drive/MyDrive/Studies/input.csv"
base_dir = "/content/drive/MyDrive/Studies"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cell 3 – βοηθητικές
def ensure_ephemeris():
    if not os.path.exists(SE_FILE):
        print("Downloading Swiss Ephemeris …")
        with urllib.request.urlopen(SE_URL) as r, open(SE_FILE, "wb") as f:
            shutil.copyfileobj(r, f)
    swe.set_ephe_path(os.getcwd())

def calc_positions(jd_ut, lat, lon):
    out = {n: swe.calc_ut(jd_ut, c, swe.FLG_SWIEPH)[0] for n, c in PLANETS.items()}
    asc, mc, *_ = swe.houses_ex(jd_ut, lat, lon, b'P')[0:7]
    out["ASC"], out["MC"] = asc, mc
    return out


In [ ]:
csv_out = os.path.join(base_dir, "angle_errors.csv")

# Cell 4 – generate one JSON per entry (Swiss Ephemeris only)
csv_path = INPUT_FILE

ensure_ephemeris()

with open(csv_path, newline='') as fin:
    reader = csv.DictReader(fin)
    for row in reader:
        # Παράδειγμα: "1979-04-18 17:30"
        dt = datetime.strptime(f"{row['date']} {row['time']}", "%Y-%m-%d %H:%M")
        jd = swe.julday(
            dt.year, dt.month, dt.day,
            dt.hour + dt.minute/60,
            swe.GREG_CAL
        )
        lat = float(row['lat'])
        lon = float(row['lon'])

        # Υπολογισμός θέσεων
        vals = calc_positions(jd, lat, lon)

        # Δημιουργία JSON
        out = {
            "id": row["id"],
            "positions_deg": vals
        }

        # Γράψε το αρχείο ακριβώς δίπλα στο CSV
        fname = os.path.join(os.path.dirname(csv_path), f"{row['id']}.json")
        with open(fname, 'w', encoding='utf8') as f:
            json.dump(out, f, ensure_ascii=False, indent=2)

        print(f"Wrote {fname}")


Wrote /content/drive/MyDrive/Studies/R-01.json
Wrote /content/drive/MyDrive/Studies/S-02.json
Wrote /content/drive/MyDrive/Studies/S-03.json
Wrote /content/drive/MyDrive/Studies/S-04.json
Wrote /content/drive/MyDrive/Studies/S-05.json
Wrote /content/drive/MyDrive/Studies/S-06.json
Wrote /content/drive/MyDrive/Studies/S-07.json
Wrote /content/drive/MyDrive/Studies/S-08.json
Wrote /content/drive/MyDrive/Studies/S-09.json
Wrote /content/drive/MyDrive/Studies/S-10.json
